## ESPP 90S Final Project: Ammonia x Wind
### Alaisha Sharma and Rowen VonPlagenhoef

In [47]:
import numpy as np
import matplotlib.pyplot as plt

### PARAMETERS

#### Wind Facility

In [48]:
num_turbine = 20                  # --
life_turbine = 20                 # yr
nameplate_turbine = 2             # MW
capacity_factor_turbine = 0.35    # --

#### Ammonia Facility

In [49]:
life_backup = 10                 # yr 
capacity_factor_backup = 0.90    # --          
backup_power_fraction = 0.1          # --
ammonia_energy_usage = 11.6      # MW-hr/ton 
ammonia_diesel_energy = 4.75     # MW-hr/ton
num_storage = 900                # --

#### Capital Costs

In [50]:
cc_land = 225e3       # $
cc_well = 30e3        # $
cc_wind = 1400        # $/kW
cc_backup = 650       # $/kW
cc_storage = 900      # $/ton
cc_ammonia = 1.3e6    # $/ton-day

#### O&M Costs

In [51]:
num_worker = 50          # --
worker_salary = 60e3     # $/yr
om_turbine = 44          # $/kW-yr
om_backup = 15           # $/kW-yrs
om_ammonia = 0.133e-3    # $/ton

#### Transportation

In [52]:
tr_hrs = 4            # hr/day
tr_distance = 150     # km/day
tr_cost = 0.07        # $/km
tr_rate = 450         # kg/hr

#### Other

In [53]:
interest_rate = 0.045    # $/yr

### FUNCTIONS

#### Annual Output

In [54]:
def annual_wind_output(params):
    # --  *  MW  *  --  *  hr/yr  =  MW-hr/yr
    return params['num_turbine']*params['nameplate_turbine']*params['capacity_factor_turbine']*8760

def annual_ammonia_output(params):
    # MW-hr/yr  *   ton/MW-hr  =  ton/yr
    return annual_wind_output(params)/params['ammonia_energy_usage']

#### Backup Generation

In [55]:
def backup_generation_size(params):
    # --  *  hr/day  =  hr/day
    hours = (1-params['capacity_factor_turbine'])*24
    # MW-hr/yr  *  yr/day  *  --  =  MW-hr/day
    energy = annual_wind_output(params)/365*params['backup_power_fraction']
    # --
    multiplier = params['life_turbine']/params['life_backup']
    # MW-hr/day  *  day/hr  *  --  =  MW
    return energy/hours*multiplier

def daily_ammonia_fuel_backup(params):
    # --  *  hr/day  =  hr/day
    hours = (1-params['capacity_factor_turbine'])*24
    # MW  *  hr/day  =  MW-hr/day
    capacity = backup_generation_size(params)*hours
    # MW-hr/day  *  ton/MW-hr  =  ton/day
    return capacity/params['ammonia_diesel_energy']

#### Capital Costs - Wind Only

In [56]:
def capital_cost_wind_only(params):
    # --  *  MW  *  $/kW  *  kW/MW  =  $
    return params['num_turbine']*params['nameplate_turbine']*params['cc_wind']*1e3

def pmt_wind_only(params):
    # $
    capital = capital_cost_wind_only(params)
    # $/yr
    return -1*np.pmt(params['interest_rate'], params['life_turbine'], capital)

def lcoe_wind_only(params):
    # $/yr  /  (MW-hr/yr  *  kW/MW)  =  $/kW-hr
    return pmt_wind_only(params)/(annual_wind_output(params)*1e3)

#### Capital Costs - Wind with Ammonia

In [57]:
def capital_cost_wind_ammonia(params):
    # $
    wind = capital_cost_wind_only(params)
    # $/kW  *  kW/MW  *  MW  =  $
    backup = params['cc_backup']*1e3*backup_generation_size(params)
    # $/ton-day  *  ton/yr  *  yr/day  =  $
    ammonia = params['cc_ammonia']*annual_ammonia_output(params)/365
    # $  *  --  =  $
    storage = params['cc_storage']*params['num_storage']
    # $
    return params['cc_land']+params['cc_well']+wind+backup+ammonia+storage
    
def pmt_wind_ammonia(params):
    # $
    capital = capital_cost_wind_ammonia(params)
    # $/yr
    return -1*np.pmt(params['interest_rate'], params['life_turbine'], capital)

def lcoe_wind_ammonia(params):
    # $/yr  /  (MW-hr/yr  *  kW/MW)  =  $/kW-hr
    return (pmt_wind_ammonia(params)+om_cost_wind_ammonia(params))/(annual_wind_output(params)*1e3)

#### O&M Costs

In [58]:
def om_cost_workers(params):
    # --  *  $/yr  = $/yr
    return params['num_worker']*params['worker_salary']

def om_cost_turbine(params):
    # MW-hr/yr
    power = annual_wind_output(params)
    # $/kW-yr  *  MW-hr/yr  *  kW/MW  *  yr/hr  =  $/yr
    return params['om_turbine']*power*1e3/8760

def om_cost_backup(params):
    # $/kW-yr  *  MW/kW  *  MW  =  $/yr
    return params['om_backup']*1e3*backup_generation_size(params)

def om_cost_ammonia(params):
    # ton/yr
    ammonia = annual_ammonia_output(params)
    # $/ton  *  ton/yr  =  $/yr
    return params['om_ammonia']*ammonia

def om_cost_wind_ammonia(params):
    # $
    return om_cost_workers(params)+om_cost_turbine(params)+om_cost_ammonia(params)+om_cost_backup(params)

#### Transportation Costs

In [59]:
def truck_transport_capacity(params):
    # kg/hr  *  ton/kg  *  hr/day  =  ton/day
    return params['tr_rate']*0.0011*params['tr_hrs']

def number_trucks(params):
    # ton/day
    capacity = truck_transport_capacity(params)
    # ton/yr
    ammonia = annual_ammonia_output(params)
    # ton/yr  *  yr/day  *  day/ton  =  --
    return ammonia/365/capacity

def transport_cost(params):
    # --
    num_truck = number_trucks(params)
    # $/km  *  km/day  *  day/yr  *  --  =  $/yr
    return params['tr_cost']*params['tr_distance']*365*num_truck

#### Final Ammonia Fuel Prices

In [60]:
def production_ammonia_price(params):
    # $/kW-hr
    lcoe = lcoe_wind_ammonia(params)
    # $/kW-hr  *  kW/MW  *  MW-hr/ton  =   $/ton
    return lcoe*1e3*params['ammonia_energy_usage']
    
def distribution_ammonia_price(params):
    # $/ton
    production = production_ammonia_price(params)
    # $/yr  *  yr/ton  =  $/ton
    transport = transport_cost(params)/annual_ammonia_output(params)
    # $/ton
    return production+transport

### CALCULATIONS

In [61]:
params_baseline = {
    'num_turbine': 20,
    'life_turbine': 20,
    'nameplate_turbine': 2,
    'capacity_factor_turbine': 0.35,
    'life_backup': 10,
    'capacity_factor_backup': 0.90,
    'backup_power_fraction': 0.1,
    'ammonia_energy_usage': 11.6,
    'ammonia_diesel_energy': 4.75,
    'num_storage': 900,
    'cc_land': 225e3,
    'cc_well': 30e3,
    'cc_wind': 1400,
    'cc_backup': 650,
    'cc_ammonia': 1.3e6,
    'cc_storage': 900,
    'num_worker': 50,
    'worker_salary': 60e3,
    'om_turbine': 44,
    'om_backup': 15,
    'om_ammonia': 0.133e-3,
    'tr_hrs': 4,
    'tr_distance': 150,
    'tr_cost': 0.07,
    'tr_rate': 450,
    'interest_rate': 0.045
}

In [77]:
print()
print("BASELINE CALCULATIONS FOR WIND + AMMONIA PLANT:")
print("-" * 47 + "\n")
print("Output of wind plant: {} MW/yr \n".format(round(annual_wind_output(params_baseline))))
print("Output of ammonia plant: {} tons/yr \n".format(round(annual_ammonia_output(params_baseline)))) 
print()
print("Size of backup generator required: {:.1f} MW \n".format(backup_generation_size(params_baseline)))
print()
print("Total capital costs of only wind plant: {:.1f} million USD \n".format(capital_cost_wind_only(params_baseline)/1e6))
print("Total capital costs of wind and ammonia plant: {:.1f} million USD\n".format(capital_cost_wind_ammonia(params_baseline)/1e6))
print()
print("PMT costs of wind plant: {:.2f} million USD/yr \n".format(pmt_wind_only(params_baseline)/1e6))
print("PMT costs of wind and ammonia plant: {:.2f} million USD/yr \n".format(pmt_wind_ammonia(params_baseline)/1e6))
print()
print("LCOE of wind plant: {:.3f} USD/kWh \n".format(lcoe_wind_only(params_baseline)))
print("LCOE of wind and ammonia plant: {:.3f} USD/yr \n".format(lcoe_wind_ammonia(params_baseline)))
print()
print("Transport cost of ammonia: {} USD/yr \n".format(round(transport_cost(params_baseline))))
print()
print("Production cost of ammonia fuel: {:.2f} $/ton \n".format(production_ammonia_price(params_baseline)))
print("Distribution cost of ammonia fuel: {:.2f} $/ton \n".format(distribution_ammonia_price(params_baseline)))


BASELINE CALCULATIONS FOR WIND + AMMONIA PLANT:
-----------------------------------------------

Output of wind plant: 122640 MW/yr 

Output of ammonia plant: 10572 tons/yr 


Size of backup generator required: 4.3 MW 


Total capital costs of only wind plant: 56.0 million USD 

Total capital costs of wind and ammonia plant: 97.5 million USD


PMT costs of wind plant: 4.31 million USD/yr 

PMT costs of wind and ammonia plant: 7.50 million USD/yr 


LCOE of wind plant: 0.035 USD/kWh 

LCOE of wind and ammonia plant: 0.091 USD/yr 


Transport cost of ammonia: 56066 USD/yr 


Production cost of ammonia fuel: 1057.24 $/ton 

Distribution cost of ammonia fuel: 1062.54 $/ton 



### GRAPHS

#### Cost Multipliers

In [ ]:
def apply_multipliers_param(params, param_names, multipliers):
    params_multiplied = []
    for m in multipliers:
        params_m = params.copy()
        for p in param_names:
            params_m[param_name] *= m
        params_multiplied.append(params_m)
    return params_multiplied

def data_with_multipliers(params_multiplied):
    multipliers = np.arange(1, 2.25, 0.25)
    wind_only = np.zeros([3, len(multipliers)])
    wind_ammonia = np.zeros([3, len(multipliers)])
    wind_only[0] = apply_multipliers(capital_cost_wind_only, params, multipliers)
    wind_only[1] = apply_multipliers(pmt_wind_only, params, multipliers)
    wind_only[2] = apply_multipliers(lcoe_wind_only, params, multipliers)
    wind_ammonia[0] = apply_multipliers(capital_cost_wind_ammonia, params, multipliers)
    wind_ammonia[1] = apply_multipliers(pmt_wind_ammonia, params, multipliers)
    wind_ammonia[2] = apply_multipliers(lcoe_wind_ammonia, params, multipliers)
    return wind_only, wind_ammonia

In [ ]:
fig, axs = plt.subplots(3, figsize=(8,14))
width = 0.25

x = np.arange(1, 2.25, 0.25)
labels = list(map(str, x))
titles = ["Total Capital", "PMT", "LCOE"]
wind_only, wind_ammonia = data_cost_multipliers(params_baseline)

for i in range(3):
    ax = axs[i]
    title = titles[i]
    ax.bar(x, wind_only[i], width/3, label='Wind')
    ax.bar(x+width/3, wind_ammonia[i], width/3, label='Wind+Ammonia')
    ax.set_ylabel('USD')
    ax.set_xticks(x)
    ax.set_xticklabels(labels)
    ax.set_title(title + " Costs with Multipliers")
    ax.legend()

fig.tight_layout()
plt.show()

#### Cost Breakdowns

In [ ]:
def capital_cost_breakdown(params, multipliers):
    capital = np.zeros([5])
    capital[0] = params['cc_land']*multipliers[0]
    capital[1] = params['cc_well']*multipliers[1]
    capital[2] = capital_cost_wind_only(params)*multipliers[2]
    capital[3] = params['cc_ammonia']*annual_ammonia_output(params)/365*multipliers[3]
    capital[4] = params['cc_storage']*params['num_storage']*multipliers[4]
    return (capital, capital/np.sum(capital))

def om_cost_breakdown(params, multipliers):
    om = np.zeros([5])
    om[0] = om_cost_workers(params)*multipliers[0]
    om[1] = om_cost_turbine(params)*multipliers[1]
    om[2] = om_cost_ammonia(params)*multipliers[2]
    om[3] = om_cost_backup(params)*multipliers[3]
    om[4] = transport_cost(params)*multipliers[4]
    return (om, om/np.sum(om))

def format_percents(percents):
    return ["{:.2f}%".format(p*1e2) for p in percents]

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, figsize=(8,12))
width = 0.35

x = np.arange(1, 6)
multipliers_capital = np.full(5, 1)
capital, capital_percents = capital_cost_breakdown(params_baseline, multipliers_capital)
multipliers_om = np.full(5, 1)
om, om_percents = om_cost_breakdown(params_baseline, multipliers_om)

labels_capital = ["Land Plot", "Well Installation", "Wind Turbines", "Ammonia", "NH3 Storage"]
ax1.bar(x, capital, width, color='b')
ax1.set_title("Capital Cost Breakdowns")
ax1.set_ylabel('Cost (USD)')
ax1.set_yscale('log')
ax1.set_xticks(x)
ax1.set_xticklabels(labels_capital)
for rect, label in zip(ax1.patches, format_percents(capital_percents)):
    height = rect.get_height()
    ax1.text(rect.get_x() + rect.get_width()/2, height*1.05, label, ha='center', va='bottom')
    
labels_om = ["Workers", "Wind Turbines", "Ammonia+Storage", "Backup Generators", "Transportation"]
ax2.bar(x, om, width, color='g')
ax2.set_title("O&M Cost Breakdowns")
ax2.set_ylabel('Cost (USD)')
ax2.set_yscale('log')
ax2.set_xticks(x)
ax2.set_xticklabels(labels_om)
for rect, label in zip(ax2.patches, format_percents(om_percents)):
    height = rect.get_height()
    ax2.text(rect.get_x() + rect.get_width()/2, height*1.05, label, ha='center', va='bottom')

fig.tight_layout()
plt.show()

#### Sensitivity Analysis

In [ ]:
def sensitivity_ammonia_prices(parameters, multipliers):
    production_prices = np.zeros([len(parameters), len(multipliers)])
    distribution_prices = np.zeros([len(parameters), len(multipliers)])
    for p in range(len(parameters)):
        params_m = params_baseline.copy()
        print(params_m)
        for m in range(len(multipliers)):
            print(parameters[p], params_m[parameters[p]], params_m[parameters[p]]*multipliers[m])
            params_m[parameters[p]] *= multipliers[m]
            production_prices[p][m] = production_ammonia_price(params_m)
            distribution_prices[p][m] = distribution_ammonia_price(params_m)
    return production_prices, distribution_prices

In [ ]:
fig, axs = plt.subplots(8, figsize=(8,24))
width = 0.25

x = np.arange(1, 2.25, 0.25)
labels = list(map(str, x))
titles = ["Well Capital", "Turbine Capital", "Ammonia Capital", "Storage Capital", "Turbine O&M", "Ammonia O&M", "Backup Generator O&M", "Transport"]
parameters = ['cc_well', 'cc_wind', 'cc_ammonia', 'cc_storage', 'om_turbine', 'om_ammonia', 'om_backup', 'tr_cost']
production_prices, distribution_prices = sensitivity_ammonia_prices(parameters, x)

for i in range(8):
    ax = axs[i]
    title = titles[i]
    ax.bar(x, production_prices[i], width/3, label='Production')
    ax.bar(x+width/3, distribution_prices[i], width/3, label='Distribution')
    ax.set_ylabel('USD')
    ax.set_xticks(x)
    ax.set_xticklabels(labels)
    ax.set_title(title + " Costs")
    ax.legend(loc='lower right')
    
fig.tight_layout()
fig.suptitle("EFFECT OF COST MULTIPLIERS")
fig.subplots_adjust(top=0.95)
plt.show()